In [11]:
# Import Dependencies
import sqlalchemy
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt


In [17]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine

engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
Base.prepare(autoload_with=engine)
print(Base.classes.keys())


[]


In [15]:
import os
print(os.path.exists("hawaii.sqlite"))  # Should print True

True


In [16]:
print(Base.classes.keys())

[]


In [ ]:

# Set up for pretty plots
%matplotlib inline

#################################################
# Database Setup
#################################################

# Create engine to connect to SQLite database
engine = create_engine("sqlite:///hawaii.sqlite")


In [19]:

# Reflect the database into a new model
Base = automap_base()
Base.prepare(autoload_with=engine)


In [20]:
# View all of the classes (tables) that were found
print(Base.classes.keys())


[]


In [21]:
import pandas as pd
import sqlite3

# Load CSVs
measurements_df = pd.read_csv("hawaii_measurements.csv")
stations_df = pd.read_csv("hawaii_stations.csv")

# Connect to a new SQLite database
conn = sqlite3.connect("hawaii.sqlite")

# Write DataFrames to the SQLite DB
measurements_df.to_sql("measurement", conn, if_exists="replace", index=False)
stations_df.to_sql("station", conn, if_exists="replace", index=False)

conn.close()

print("Database created and tables loaded successfully.")


FileNotFoundError: [Errno 2] No such file or directory: 'hawaii_measurements.csv'

In [14]:

# Save references to tables
Measurement = Base.classes.Measurement
Station = Base.classes.Station

# Create Session
session = Session(engine)

#################################################
# Climate Analysis and Data Exploration
#################################################

# Find the most recent date in the dataset
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(f"Most recent date in the dataset: {recent_date[0]}")

# Calculate the date one year ago from the most recent date
last_date = dt.datetime.strptime(recent_date[0], "%Y-%m-%d")
one_year_ago = last_date - dt.timedelta(days=365)

# Perform the query for the last 12 months of precipitation data
precipitation_data = session.query(Measurement.date, Measurement.prcp)\
    .filter(Measurement.date >= one_year_ago)\
    .order_by(Measurement.date).all()

# Convert the query results to a Pandas DataFrame
precip_df = pd.DataFrame(precipitation_data, columns=["Date", "Precipitation"])

# Set the index to the date column
precip_df.set_index("Date", inplace=True)

# Display the first few rows
precip_df.head()

# Plot the precipitation data
precip_df.plot(figsize=(12,6))
plt.xlabel("Date")
plt.ylabel("Precipitation (inches)")
plt.title("Daily Precipitation - Last 12 Months")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Summary statistics for precipitation data
precip_df.describe()

# Number of stations available in the dataset
station_count = session.query(Station.station).count()
print(f"Number of stations: {station_count}")

# List stations and their observation counts
station_activity = session.query(Measurement.station, func.count(Measurement.id))\
    .group_by(Measurement.station)\
    .order_by(func.count(Measurement.id).desc()).all()

station_activity

# Most active station
most_active_station = station_activity[0][0]
print(f"Most active station: {most_active_station}")

# Temperature statistics (min, avg, max) for the most active station
temp_stats = session.query(func.min(Measurement.tobs),
                           func.avg(Measurement.tobs),
                           func.max(Measurement.tobs))\
                    .filter(Measurement.station == most_active_station).all()

print(f"Min Temp: {temp_stats[0][0]}, Avg Temp: {temp_stats[0][1]}, Max Temp: {temp_stats[0][2]}")

# Query the last 12 months of temperature observation data for the most active station
temp_data = session.query(Measurement.date, Measurement.tobs)\
    .filter(Measurement.date >= one_year_ago)\
    .filter(Measurement.station == most_active_station)\
    .order_by(Measurement.date).all()

# Convert to DataFrame
temp_df = pd.DataFrame(temp_data, columns=["Date", "Temperature"])
temp_df.set_index("Date", inplace=True)

temp_df.head()

# Plot a histogram of the temperature observations
temp_df.plot.hist(bins=12, figsize=(10,6))
plt.xlabel("Temperature (F)")
plt.title("Temperature Observations (Most Active Station)")
plt.show()

# Reflect the database into a new model
Base = automap_base()
Base.prepare(autoload_with=engine)

# Print the table names SQLAlchemy found
print(Base.classes.keys())


AttributeError: Measurement

In [ ]:

# Create Session
session = Session(engine)

#################################################
# Climate Analysis and Data Exploration
#################################################

# Find the most recent date in the dataset
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(f"Most recent date in the dataset: {recent_date[0]}")

# Calculate the date one year ago from the most recent date
last_date = dt.datetime.strptime(recent_date[0], "%Y-%m-%d")
one_year_ago = last_date - dt.timedelta(days=365)

# Perform the query for the last 12 months of precipitation data
precipitation_data = session.query(Measurement.date, Measurement.prcp)\
    .filter(Measurement.date >= one_year_ago)\
    .order_by(Measurement.date).all()

# Convert the query results to a Pandas DataFrame
precip_df = pd.DataFrame(precipitation_data, columns=["Date", "Precipitation"])

# Set the index to the date column
precip_df.set_index("Date", inplace=True)

# Display the first few rows
precip_df.head()

# Plot the precipitation data
precip_df.plot(figsize=(12,6))
plt.xlabel("Date")
plt.ylabel("Precipitation (inches)")
plt.title("Daily Precipitation - Last 12 Months")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Summary statistics for precipitation data
precip_df.describe()

# Number of stations available in the dataset
station_count = session.query(Station.station).count()
print(f"Number of stations: {station_count}")

# List stations and their observation counts
station_activity = session.query(Measurement.station, func.count(Measurement.id))\
    .group_by(Measurement.station)\
    .order_by(func.count(Measurement.id).desc()).all()

station_activity

# Most active station
most_active_station = station_activity[0][0]
print(f"Most active station: {most_active_station}")

# Temperature statistics (min, avg, max) for the most active station
temp_stats = session.query(func.min(Measurement.tobs),
                           func.avg(Measurement.tobs),
                           func.max(Measurement.tobs))\
                    .filter(Measurement.station == most_active_station).all()

print(f"Min Temp: {temp_stats[0][0]}, Avg Temp: {temp_stats[0][1]}, Max Temp: {temp_stats[0][2]}")

# Query the last 12 months of temperature observation data for the most active station
temp_data = session.query(Measurement.date, Measurement.tobs)\
    .filter(Measurement.date >= one_year_ago)\
    .filter(Measurement.station == most_active_station)\
    .order_by(Measurement.date).all()

# Convert to DataFrame
temp_df = pd.DataFrame(temp_data, columns=["Date", "Temperature"])
temp_df.set_index("Date", inplace=True)

temp_df.head()

# Plot a histogram of the temperature observations
temp_df.plot.hist(bins=12, figsize=(10,6))
plt.xlabel("Temperature (F)")
plt.title("Temperature Observations (Most Active Station)")
plt.show()

# Reflect the database into a new model
Base = automap_base()
Base.prepare(autoload_with=engine)

# Print the table names SQLAlchemy found
print(Base.classes.keys())
